# Library

In [2]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

# File Load
- 4등급_차량_리스트_양식_result
- 정기
- 정밀
- 3등급 등록&제원
- 5등급 등록&제원

In [3]:
ods_fold = 'D:/data/big2/ODS'
fold = 'D:/data/big2/BD1_4등급_데이터_점검1'

In [4]:
# about 11s
file_name = '4등급_차량_리스트_양식_result.csv'
df = pd.read_csv(os.path.join(fold, file_name), encoding='cp949', low_memory=False)

In [5]:
# about 3m 33s
file_name = '[ODS]정기검사(2022.12.29).csv'
ins = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)

In [6]:
# about 5m 55s
file_name = '[ODS]정밀검사(2022.12.29).csv'
dins = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)

In [130]:
# about 2m 26s
file_name = '[ODS][G3]등록정보&제원정보(말소제거)(2022.12.29).csv'
g3 = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)

In [131]:
# about 18s
file_name = '[ODS][G5]등록정보&제원정보(말소제거)(2022.12.29).csv'
g5 = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)

In [132]:
df.shape, ins.shape, dins.shape, g3.shape, g5.shape

((1153813, 28), (22006426, 34), (28682228, 34), (8759431, 49), (1154762, 49))

In [8]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무'],
      dtype='object')

In [9]:
ins.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간'],
      dtype='object')

In [10]:
dins.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간',
       '검사 검사소코드', '검사유효기간'],
      dtype='object')

In [133]:
g3.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'],
      dtype='object')

In [134]:
g5.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'],
      dtype='object')

# Preprocessing

## 전처리 : 4등급_차량_리스트_양식_result
- 컬럼명 정리
    - '차종' : '차종_원본', '인증번호':'배출가스인증번호'
- ~~중복 차대번호 처리 : '최초등록일', '검사유효일, 내림차순 정렬(ascending=[False, False])~~
- 배출가스등급 : 4등급
- 연료 : 경유차

In [11]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무'],
      dtype='object')

In [12]:
df = df.rename(columns={'차종':'차종_원본', '인증번호':'배출가스인증번호'})
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무'],
      dtype='object')

In [13]:
df.shape, len(df['차대번호'].unique())

((1502235, 28), 1502213)

In [14]:
# df = df.sort_values(['최초등록일', '검사유효일'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
# df.shape

In [15]:
df['등급'].value_counts(dropna=False)

4    1502235
Name: 등급, dtype: int64

In [16]:
df['연료'].value_counts(dropna=False)

경유             1153813
휘발유             322983
LPG(액화석유가스)      25400
기타연료                37
알코올                  1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [17]:
df.shape

(1502235, 28)

In [18]:
df = df[df['연료'] == '경유'].reset_index(drop=True)
df.shape

(1153813, 28)

## 전처리 : 정기, 정밀 검사
- 정기, 정밀 검사 필요한 컬럼만 추출
- 정기, 정밀 검사 샘플에 '정기, 정밀' 검사 표기
- 정기, 정밀 통합하여 중복 차대번호 제거(최신 '검사일자'만 사용)

In [19]:
len(ins.columns), len(dins.columns)

(39, 41)

In [20]:
col = ['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6',]
len(col)

33

In [21]:
# about 30s
ins = ins[col]
dins = dins[col]
ins.shape, dins.shape

((22006426, 33), (28682228, 33))

In [22]:
ins['정기정밀'] = '정기'
dins['정기정밀'] = '정밀'

In [23]:
# about 13s
ins.shape, len(ins['차대번호'].unique())

((22006426, 34), 12601262)

In [24]:
# 20s
dins.shape, len(dins['차대번호'].unique())

((28682228, 34), 15367596)

In [25]:
# about 
tin = pd.concat([ins, dins], ignore_index=True)
tin.shape

(50688654, 34)

In [26]:
tin.shape[0] == ins.shape[0] + dins.shape[0]

True

In [27]:
tin = tin.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
tin.shape

(22044208, 34)

## 전처리 : 등록&제원과 정기,정밀 병합
- 4등급 경유 등록&제원 : df
- 최신 검사일자 중복 차대번호 제거 정기, 정밀 : tin
- 병합 : total

In [99]:
# about 45s
total = df.merge(tin, on='차대번호', how='left')
total.shape

(1153813, 61)

## 전처리 : 배인번호 빈칸, 빈칸 포함 문자열 처리

In [100]:
total['배출가스인증번호'].isnull().sum()

7147

In [ ]:
count = 0
for one in total['배출가스인증번호']:
    if ' ' == str(one) or ' ' in str(one):
        count += 1
count

In [101]:
count = 0
for one in total['배출가스인증번호']:
    if ' ' == str(one):
        count += 1
count

3

In [102]:
idx = []
for i, one in enumerate(total['배출가스인증번호']):
    if ' ' == str(one):
        idx.append(i)
len(idx)

3

In [103]:
total.loc[idx, '배출가스인증번호'] = np.nan

In [104]:
count = 0
for one in total['배출가스인증번호']:
    if ' ' == str(one):
        count += 1
count

0

In [105]:
count = 0
for one in total['배출가스인증번호']:
    if ' ' in str(one):
        count += 1
count

10

In [106]:
total['배출가스인증번호'] = total['배출가스인증번호'].str.strip()

In [108]:
count = 0
for one in total['배출가스인증번호']:
    if ' ' in str(one):
        count += 1
count

0

In [109]:
count = 0
for one in total['배출가스인증번호']:
    if ' ' == str(one) or ' ' in str(one):
        count += 1
count

0

## 전처리 : 빈값 문자열 형식으로 변환

In [110]:
total['배출가스인증번호'].value_counts(dropna=False)

7MY-HD-14-68     252249
8MY-KM-13-19      90619
6MY-KM-14-61      71965
9MY-KM-14-41      65879
7MY-HD-14-46      58125
                  ...  
8MY-EC-0M-24          1
8MY-DH-0M-24          1
9MY-PP-14-409         1
7MY-MM-0M-24          1
8MY-FM-0M-24          1
Name: 배출가스인증번호, Length: 206, dtype: int64

In [111]:
total['배출가스인증번호'].isnull().sum()

7150

In [112]:
total['검사방법'].value_counts(dropna=False)

부하검사(KD-147)        798238
무부하검사(급가속)          289263
부하검사(LUG DOWN)       43669
NaN                  22637
부하검사(ASM-Idling)         3
무부하검사(정지가동)              2
무부하검사(TSI)               1
Name: 검사방법, dtype: int64

In [113]:
total['무부하매연판정1'].value_counts(dropna=False)

Y      1087888
N        34609
NaN      31316
Name: 무부하매연판정1, dtype: int64

In [114]:
total['무부하매연측정치1'].value_counts(dropna=False)

0.00      276379
1.00       84223
2.00       70398
3.00       63318
4.00       62680
           ...  
0.01           1
99.90          1
2.50           1
135.00         1
103.00         1
Name: 무부하매연측정치1, Length: 111, dtype: int64

In [115]:
total['무부하매연측정치1'].isnull().sum()

31312

In [116]:
total['검사방법'] = total['검사방법'].astype('str')
total['무부하매연판정1'] = total['무부하매연판정1'].astype('str')
total['배출가스인증번호'] = total['배출가스인증번호'].astype('str')

In [117]:
total['검사방법'].value_counts()

부하검사(KD-147)        798238
무부하검사(급가속)          289263
부하검사(LUG DOWN)       43669
nan                  22637
부하검사(ASM-Idling)         3
무부하검사(정지가동)              2
무부하검사(TSI)               1
Name: 검사방법, dtype: int64

In [118]:
total['무부하매연판정1'].value_counts()

Y      1087888
N        34609
nan      31316
Name: 무부하매연판정1, dtype: int64

In [119]:
total['배출가스인증번호'].isnull().sum()

0

## 전처리 : 3등급 중복 차대번호 제거

In [135]:
g3.shape, len(g3['차대번호'].unique())

((8759431, 49), 8759393)

In [136]:
g3 = g3.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)

In [139]:
g3.shape

(8759393, 49)

## 전처리 : 5등급 중복 차대번호 제거

In [137]:
g5.shape, len(g5['차대번호'].unique())

((1154762, 49), 1154724)

In [138]:
g5 = g5.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)

In [140]:
g5.shape

(1154724, 49)

## 전처리 : 4등급 차대번호 10자리 연식 추출

In [141]:
total['vin10']  = total['차대번호'].str[9]
total[['차대번호', 'vin10']].head()

,차대번호,vin10
0,KNAMM816BBS461795,B
1,KPACE4AB19P052332,9
2,KNAKU815BAA068990,A
3,KL3M8BNF37K000098,7
4,KNHMD376BAS393884,A


In [142]:
total['vin10'].isnull().sum(), total['vin10'].dtype

(0, dtype('O'))

In [143]:
vin10_dict = {'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009, 'A':2010, 'B':2011, 'C':2012, 'D':2013, 'E':2014, 'F':2015, 'G':2016, 'H':2017, 'J':2018, 'K':2019, 'L':2020}

In [144]:
total['vin_year'] = total['vin10'].map(vin10_dict)
total[['차대번호', 'vin10', 'vin_year']].head()

,차대번호,vin10,vin_year
0,KNAMM816BBS461795,B,2011.0
1,KPACE4AB19P052332,9,2009.0
2,KNAKU815BAA068990,A,2010.0
3,KL3M8BNF37K000098,7,2007.0
4,KNHMD376BAS393884,A,2010.0


# Analysis
- '배출가스인증번호'별 '무부하매연판정1'별 '차대번호' 개수
- '배출가스인증번호'별 '검사방법', '무부하매연판정1'별 '차대번호' 개수
- '배출가스인증번호'별 '검사방법', '무부하매연판정1'별 '무부하매연측정치1'의 통계

## 차대번호 이상 확인
- 17자리 아님 : 10개
- 중복 없음

In [167]:
vin_idx = []
for i, one in enumerate(total['차대번호']):
    if len(one) != 17:
        vin_idx.append(i)
len(vin_idx)

10

In [168]:
total.loc[vin_idx, '차대번호']

232819     KNAJC52438A78075
281813     KNCSJX4AAK452185
299478     KMFZC7JP9U519465
400425     KMFZC7JP9U481571
467880     KMFWBX7ABU267970
828838     KMFZCS7JP8U43881
926825     KLTBT9CM29K00095
928545     WAUZZZ4L87D67998
953328     KMFZCZ7JAB640917
1030687    KNME5C2MM8P00423
Name: 차대번호, dtype: object

In [169]:
total.shape, len(total['차대번호'].unique())

((1153813, 63), 1153813)

## 차량연식과 vin10자리 연식 비교
- 63개 다름

In [170]:
total.columns

Index(['차대번호', '차량번호_x', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자',
       '검사유효일', '배출가스인증번호', '등급', '차명_x', '차종분류', '차종유형', '자동차형식', '제작사명',
       '연료', '엔진형식_x', '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명',
       '소유자구분', '말소여부', '일치유무', '사용연료', '주행거리', '차명_y', '차량번호_y', '차종', '차량연식',
       '엔진형식_y', '검사방법', '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값',
       '무부하매연판정1', '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6',
       '무부하매연허용치1', '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5',
       '무부하매연허용치6', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4',
       '무부하매연측정치5', '무부하매연측정치6', '정기정밀', 'vin10', 'vin_year'],
      dtype='object')

In [173]:
total[total['연식'] != total['vin_year']].shape

(63, 63)

In [175]:
total.loc[total['연식'] != total['vin_year'], ['연식', 'vin10', 'vin_year']]

,연식,vin10,vin_year
3,2008,7,2007.0
4943,2010,9,2009.0
7088,2007,8,2008.0
24336,2007,8,2008.0
37918,2010,B,2011.0
...,...,...,...
1094374,2009,8,2008.0
1095546,2010,B,2011.0
1103828,2010,9,2009.0
1120030,2011,A,2010.0


## 배인번호별_검사방법별_매연측정치_통계

In [120]:
total.groupby(['무부하매연판정1'], dropna=False)['차대번호'].agg(['count'])

,count
무부하매연판정1,
N,34609
Y,1087888
nan,31316


In [121]:
gas_total_group = total.groupby(['무부하매연판정1', '검사방법'], dropna=False)['차대번호'].agg(['count']).unstack()
gas_total_group

count                                                     \
검사방법          nan 무부하검사(TSI) 무부하검사(급가속) 무부하검사(정지가동) 부하검사(ASM-Idling)   
무부하매연판정1                                                               
N             NaN        NaN     5724.0         NaN              NaN   
Y             NaN        1.0   283244.0         2.0              3.0   
nan       22637.0        NaN      295.0         NaN              NaN   

                                      
검사방법     부하검사(KD-147) 부하검사(LUG DOWN)  
무부하매연판정1                              
N             27614.0         1271.0  
Y            762382.0        42256.0  
nan            8242.0          142.0

In [122]:
gas_grp = total.groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['무부하매연측정치1'].agg(['count', 'mean', 'min', 'max']).stack().unstack('검사방법').unstack()
gas_grp

검사방법                    nan              무부하검사(TSI)                무부하검사(급가속)  \
                      count mean min max      count mean  min  max      count   
배출가스인증번호     무부하매연판정1                                                           
4MY-AD-20    N          NaN  NaN NaN NaN        NaN  NaN  NaN  NaN        1.0   
             Y          NaN  NaN NaN NaN        NaN  NaN  NaN  NaN       26.0   
             nan        0.0  NaN NaN NaN        NaN  NaN  NaN  NaN        NaN   
4MY-DC-33    Y          NaN  NaN NaN NaN        NaN  NaN  NaN  NaN        2.0   
             nan        0.0  NaN NaN NaN        NaN  NaN  NaN  NaN        NaN   
...                     ...  ...  ..  ..        ...  ...  ...  ...        ...   
FMY-VTK-14-3 nan        0.0  NaN NaN NaN        NaN  NaN  NaN  NaN        NaN   
FMY-VTK-14-4 Y          NaN  NaN NaN NaN        NaN  NaN  NaN  NaN       25.0   
nan          N          NaN  NaN NaN NaN        NaN  NaN  NaN  NaN        6.0   
             Y          NaN  NaN NaN NaN        1.0  0.0  0.0  0.0     1939.0   
             nan        0.0  NaN NaN NaN        NaN  NaN  NaN  NaN        0.0   

검사방법                              ... 부하검사(ASM-Idling)       부하검사(KD-147)  \
                            mean  ...              min   max        count   
배출가스인증번호     무부하매연판정1             ...                                       
4MY-AD-20    N         51.000000  ...              NaN   NaN          NaN   
             Y         13.269231  ...              NaN   NaN          NaN   
             nan             NaN  ...              NaN   NaN          NaN   
4MY-DC-33    Y          0.000000  ...              NaN   NaN          NaN   
             nan             NaN  ...              NaN   NaN          NaN   
...                          ...  ...              ...   ...          ...   
FMY-VTK-14-3 nan             NaN  ...              NaN   NaN          NaN   
FMY-VTK-14-4 Y          1.360000  ...              NaN   NaN          NaN   
nan          N         43.833333  ...              NaN   NaN         96.0   
             Y          2.832388  ...             0.03  0.03       4391.0   
             nan             NaN  ...              NaN   NaN          0.0   

검사방법                                         부하검사(LUG DOWN)                   \
                            mean   min   max          count       mean   min   
배출가스인증번호     무부하매연판정1                                                          
4MY-AD-20    N               NaN   NaN   NaN            NaN        NaN   NaN   
             Y               NaN   NaN   NaN            NaN        NaN   NaN   
             nan             NaN   NaN   NaN            NaN        NaN   NaN   
4MY-DC-33    Y               NaN   NaN   NaN            4.0   9.000000   0.0   
             nan             NaN   NaN   NaN            NaN        NaN   NaN   
...                          ...   ...   ...            ...        ...   ...   
FMY-VTK-14-3 nan             NaN   NaN   NaN            NaN        NaN   NaN   
FMY-VTK-14-4 Y               NaN   NaN   NaN           24.0   0.458333   0.0   
nan          N         36.395833  10.0  94.0            3.0  17.666667  17.0   
             Y          3.682305   0.0  25.0          305.0   3.442623   0.0   
             nan             NaN   NaN   NaN            NaN        NaN   NaN   

검사방법                         
                        max  
배출가스인증번호     무부하매연판정1        
4MY-AD-20    N          NaN  
             Y          NaN  
             nan        NaN  
4MY-DC-33    Y         20.0  
             nan        NaN  
...                     ...  
FMY-VTK-14-3 nan        NaN  
FMY-VTK-14-4 Y          4.0  
nan          N         19.0  
             Y         26.0  
             nan        NaN  

[452 rows x 28 columns]

In [123]:
gas_vin_grp = total.groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['차대번호'].agg(['count']).unstack()
gas_vin_grp

count                                    \
검사방법                     nan 무부하검사(TSI) 무부하검사(급가속) 무부하검사(정지가동)   
배출가스인증번호     무부하매연판정1                                            
4MY-AD-20    N           NaN        NaN        1.0         NaN   
             Y           NaN        NaN       26.0         NaN   
             nan         2.0        NaN        NaN         NaN   
4MY-DC-33    Y           NaN        NaN        2.0         NaN   
             nan         1.0        NaN        NaN         NaN   
...                      ...        ...        ...         ...   
FMY-VTK-14-3 nan         2.0        NaN        NaN         NaN   
FMY-VTK-14-4 Y           NaN        NaN       25.0         NaN   
nan          N           NaN        NaN        6.0         NaN   
             Y           NaN        1.0     1939.0         NaN   
             nan       378.0        NaN        5.0         NaN   

                                                                    
검사방법                  부하검사(ASM-Idling) 부하검사(KD-147) 부하검사(LUG DOWN)  
배출가스인증번호     무부하매연판정1                                               
4MY-AD-20    N                     NaN          NaN            NaN  
             Y                     NaN          NaN            NaN  
             nan                   NaN          NaN            NaN  
4MY-DC-33    Y                     NaN          NaN            4.0  
             nan                   NaN          NaN            NaN  
...                                ...          ...            ...  
FMY-VTK-14-3 nan                   NaN          NaN            NaN  
FMY-VTK-14-4 Y                     NaN          NaN           24.0  
nan          N                     NaN         96.0            3.0  
             Y                     1.0       4391.0          305.0  
             nan                   NaN         25.0            NaN  

[452 rows x 7 columns]

In [124]:
gas_grp_y = total[total['무부하매연판정1'] == 'Y'].groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['무부하매연측정치1'].agg(['count', 'mean', 'min', 'max']).stack().unstack('검사방법').unstack()
gas_grp_y

검사방법                  무부하검사(TSI)                무부하검사(급가속)                  \
                           count mean  min  max      count       mean  min   
배출가스인증번호     무부하매연판정1                                                        
4MY-AD-20    Y               NaN  NaN  NaN  NaN       26.0  13.269231  0.0   
4MY-DC-33    Y               NaN  NaN  NaN  NaN        2.0   0.000000  0.0   
4MY-PA-06    Y               NaN  NaN  NaN  NaN       12.0  14.500000  0.0   
5MY-AD-13-06 Y               NaN  NaN  NaN  NaN      311.0   1.202572  0.0   
5MY-AD-13-07 Y               NaN  NaN  NaN  NaN      108.0   1.712963  0.0   
...                          ...  ...  ...  ...        ...        ...  ...   
FMY-VTK-14-1 Y               NaN  NaN  NaN  NaN      337.0   0.952522  0.0   
FMY-VTK-14-2 Y               NaN  NaN  NaN  NaN       13.0   0.000000  0.0   
FMY-VTK-14-3 Y               NaN  NaN  NaN  NaN       11.0   0.000000  0.0   
FMY-VTK-14-4 Y               NaN  NaN  NaN  NaN       25.0   1.360000  0.0   
nan          Y               1.0  0.0  0.0  0.0     1939.0   2.832388  0.0   

검사방법                        무부하검사(정지가동)       ... 부하검사(ASM-Idling)        \
                        max       count mean  ...              min   max   
배출가스인증번호     무부하매연판정1                         ...                          
4MY-AD-20    Y         31.0         NaN  NaN  ...              NaN   NaN   
4MY-DC-33    Y          0.0         NaN  NaN  ...              NaN   NaN   
4MY-PA-06    Y         40.0         NaN  NaN  ...              NaN   NaN   
5MY-AD-13-06 Y         27.0         NaN  NaN  ...              NaN   NaN   
5MY-AD-13-07 Y         45.0         NaN  NaN  ...              NaN   NaN   
...                     ...         ...  ...  ...              ...   ...   
FMY-VTK-14-1 Y         19.0         NaN  NaN  ...              NaN   NaN   
FMY-VTK-14-2 Y          0.0         NaN  NaN  ...              NaN   NaN   
FMY-VTK-14-3 Y          0.0         NaN  NaN  ...              NaN   NaN   
FMY-VTK-14-4 Y         21.0         NaN  NaN  ...              NaN   NaN   
nan          Y         41.0         NaN  NaN  ...             0.03  0.03   

검사방법                  부하검사(KD-147)                      부하검사(LUG DOWN)  \
                             count      mean  min   max          count   
배출가스인증번호     무부하매연판정1                                                    
4MY-AD-20    Y                 NaN       NaN  NaN   NaN            NaN   
4MY-DC-33    Y                 NaN       NaN  NaN   NaN            4.0   
4MY-PA-06    Y                 NaN       NaN  NaN   NaN            NaN   
5MY-AD-13-06 Y                 NaN       NaN  NaN   NaN            NaN   
5MY-AD-13-07 Y                 NaN       NaN  NaN   NaN            NaN   
...                            ...       ...  ...   ...            ...   
FMY-VTK-14-1 Y                 NaN       NaN  NaN   NaN         1419.0   
FMY-VTK-14-2 Y                 NaN       NaN  NaN   NaN           73.0   
FMY-VTK-14-3 Y                 NaN       NaN  NaN   NaN           63.0   
FMY-VTK-14-4 Y                 NaN       NaN  NaN   NaN           24.0   
nan          Y              4391.0  3.682305  0.0  25.0          305.0   

검사방법                                        
                           mean  min   max  
배출가스인증번호     무부하매연판정1                       
4MY-AD-20    Y              NaN  NaN   NaN  
4MY-DC-33    Y         9.000000  0.0  20.0  
4MY-PA-06    Y              NaN  NaN   NaN  
5MY-AD-13-06 Y              NaN  NaN   NaN  
5MY-AD-13-07 Y              NaN  NaN   NaN  
...                         ...  ...   ...  
FMY-VTK-14-1 Y         0.879493 -1.0  15.0  
FMY-VTK-14-2 Y         0.671233  0.0  10.0  
FMY-VTK-14-3 Y         0.238095  0.0   6.0  
FMY-VTK-14-4 Y         0.458333  0.0   4.0  
nan          Y         3.442623  0.0  26.0  

[205 rows x 24 columns]

In [125]:
gas_grp_y_vin = total[total['무부하매연판정1'] == 'Y'].groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['차대번호'].agg('count').unstack()
gas_grp_y_vin

,검사방법,무부하검사(TSI),무부하검사(급가속),무부하검사(정지가동),부하검사(ASM-Idling),부하검사(KD-147),부하검사(LUG DOWN)
배출가스인증번호,무부하매연판정1,,,,,,
4MY-AD-20,Y,NaN,26.0,NaN,NaN,NaN,NaN
4MY-DC-33,Y,NaN,2.0,NaN,NaN,NaN,4.0
4MY-PA-06,Y,NaN,12.0,NaN,NaN,NaN,NaN
5MY-AD-13-06,Y,NaN,311.0,NaN,NaN,NaN,NaN
5MY-AD-13-07,Y,NaN,108.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
FMY-VTK-14-1,Y,NaN,337.0,NaN,NaN,NaN,1419.0
FMY-VTK-14-2,Y,NaN,13.0,NaN,NaN,NaN,73.0
FMY-VTK-14-3,Y,NaN,11.0,NaN,NaN,NaN,63.0


In [126]:
gas_grp_n = total[total['무부하매연판정1'] == 'N'].groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['무부하매연측정치1'].agg(['count', 'mean', 'min', 'max']).stack().unstack('검사방법').unstack()
gas_grp_n

검사방법                  무부하검사(급가속)                        부하검사(KD-147)  \
                           count       mean   min   max        count   
배출가스인증번호     무부하매연판정1                                                  
4MY-AD-20    N               1.0  51.000000  51.0  51.0          NaN   
5MY-AD-13-06 N               1.0  51.000000  51.0  51.0          NaN   
5MY-AD-14-08 N               NaN        NaN   NaN   NaN          1.0   
5MY-AD-14-18 N               1.0  45.000000  45.0  45.0          3.0   
5MY-HB-24-02 N               NaN        NaN   NaN   NaN          5.0   
...                          ...        ...   ...   ...          ...   
9MY-KM-14-41 N             162.0  43.469136  21.0  99.0       2081.0   
9MY-PA-13-1  N               1.0  22.000000  22.0  22.0          NaN   
C135         N               1.0  59.000000  59.0  59.0          1.0   
FMY-VTK-14-1 N               NaN        NaN   NaN   NaN          NaN   
nan          N               6.0  43.833333  16.0  91.0         96.0   

검사방법                                         부하검사(LUG DOWN)                   \
                            mean   min   max          count       mean   min   
배출가스인증번호     무부하매연판정1                                                          
4MY-AD-20    N               NaN   NaN   NaN            NaN        NaN   NaN   
5MY-AD-13-06 N               NaN   NaN   NaN            NaN        NaN   NaN   
5MY-AD-14-08 N         65.000000  65.0  65.0            NaN        NaN   NaN   
5MY-AD-14-18 N         31.666667  20.0  49.0            NaN        NaN   NaN   
5MY-HB-24-02 N         48.600000  30.0  80.0            NaN        NaN   NaN   
...                          ...   ...   ...            ...        ...   ...   
9MY-KM-14-41 N         41.281115  16.0  99.0            1.0  21.000000  21.0   
9MY-PA-13-1  N               NaN   NaN   NaN            NaN        NaN   NaN   
C135         N         53.000000  53.0  53.0            NaN        NaN   NaN   
FMY-VTK-14-1 N               NaN   NaN   NaN            4.0  24.750000  18.0   
nan          N         36.395833  10.0  94.0            3.0  17.666667  17.0   

검사방법                         
                        max  
배출가스인증번호     무부하매연판정1        
4MY-AD-20    N          NaN  
5MY-AD-13-06 N          NaN  
5MY-AD-14-08 N          NaN  
5MY-AD-14-18 N          NaN  
5MY-HB-24-02 N          NaN  
...                     ...  
9MY-KM-14-41 N         21.0  
9MY-PA-13-1  N          NaN  
C135         N          NaN  
FMY-VTK-14-1 N         38.0  
nan          N         19.0  

[105 rows x 12 columns]

In [127]:
gas_grp_n_vin = total[total['무부하매연판정1'] == 'N'].groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['차대번호'].agg('count').unstack()
gas_grp_n_vin

,검사방법,무부하검사(급가속),부하검사(KD-147),부하검사(LUG DOWN)
배출가스인증번호,무부하매연판정1,,,
4MY-AD-20,N,1.0,NaN,NaN
5MY-AD-13-06,N,1.0,NaN,NaN
5MY-AD-14-08,N,NaN,1.0,NaN
5MY-AD-14-18,N,1.0,3.0,NaN
5MY-HB-24-02,N,NaN,5.0,NaN
...,...,...,...,...
9MY-KM-14-41,N,162.0,2081.0,1.0
9MY-PA-13-1,N,1.0,NaN,NaN
C135,N,1.0,1.0,NaN


In [128]:
pd.merge(gas_grp_y.reset_index(), gas_grp_n.reset_index(), on='배출가스인증번호', how='outer')

C:\Users\kbjung\AppData\Local\Temp\ipykernel_9420\183258868.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pd.merge(gas_grp_y.reset_index(), gas_grp_n.reset_index(), on='배출가스인증번호', how='outer')


검사방법      배출가스인증번호 무부하매연판정1_x 무부하검사(TSI)                무부하검사(급가속)_x  \
                                   count mean  min  max        count   
0        4MY-AD-20          Y        NaN  NaN  NaN  NaN         26.0   
1        4MY-DC-33          Y        NaN  NaN  NaN  NaN          2.0   
2        4MY-PA-06          Y        NaN  NaN  NaN  NaN         12.0   
3     5MY-AD-13-06          Y        NaN  NaN  NaN  NaN        311.0   
4     5MY-AD-13-07          Y        NaN  NaN  NaN  NaN        108.0   
..             ...        ...        ...  ...  ...  ...          ...   
201   FMY-VTK-14-2          Y        NaN  NaN  NaN  NaN         13.0   
202   FMY-VTK-14-3          Y        NaN  NaN  NaN  NaN         11.0   
203   FMY-VTK-14-4          Y        NaN  NaN  NaN  NaN         25.0   
204            nan          Y        1.0  0.0  0.0  0.0       1939.0   
205   8MY-BD-0M-24        NaN        NaN  NaN  NaN  NaN          NaN   

검사방법                        ... 무부하검사(급가속)_y       부하검사(KD-147)_y             \
           mean  min   max  ...          min   max          count       mean   
0     13.269231  0.0  31.0  ...         51.0  51.0            NaN        NaN   
1      0.000000  0.0   0.0  ...          NaN   NaN            NaN        NaN   
2     14.500000  0.0  40.0  ...          NaN   NaN            NaN        NaN   
3      1.202572  0.0  27.0  ...         51.0  51.0            NaN        NaN   
4      1.712963  0.0  45.0  ...          NaN   NaN            NaN        NaN   
..          ...  ...   ...  ...          ...   ...            ...        ...   
201    0.000000  0.0   0.0  ...          NaN   NaN            NaN        NaN   
202    0.000000  0.0   0.0  ...          NaN   NaN            NaN        NaN   
203    1.360000  0.0  21.0  ...          NaN   NaN            NaN        NaN   
204    2.832388  0.0  41.0  ...         16.0  91.0           96.0  36.395833   
205         NaN  NaN   NaN  ...          NaN   NaN            NaN        NaN   

검사방법             부하검사(LUG DOWN)_y                         
       min   max            count       mean   min   max  
0      NaN   NaN              NaN        NaN   NaN   NaN  
1      NaN   NaN              NaN        NaN   NaN   NaN  
2      NaN   NaN              NaN        NaN   NaN   NaN  
3      NaN   NaN              NaN        NaN   NaN   NaN  
4      NaN   NaN              NaN        NaN   NaN   NaN  
..     ...   ...              ...        ...   ...   ...  
201    NaN   NaN              NaN        NaN   NaN   NaN  
202    NaN   NaN              NaN        NaN   NaN   NaN  
203    NaN   NaN              NaN        NaN   NaN   NaN  
204   10.0  94.0              3.0  17.666667  17.0  19.0  
205    NaN   NaN              1.0  16.000000  16.0  16.0  

[206 rows x 39 columns]

## Export [BD1][통계][G4]배인번호별_검사방법별_매연측정치_통계

In [129]:
with pd.ExcelWriter('analysis2/4등급_데이터_점검/[BD1][통계][G4]배인번호별_검사방법별_매연측정치_통계.xlsx') as writer:
    # 전체 합격/불합격별 대수 통계
    gas_total_group.to_excel(writer, sheet_name='판정별_통계')
    # 배출가스인증번호, 판정별 매연측정치 통계
    gas_grp.to_excel(writer, sheet_name = '매연측정치_통계')
    gas_vin_grp.to_excel(writer, sheet_name = '매연측정치_차대번호_통계')
    # 매연판정 합격/불합격별 매연측정치 통계
    gas_grp_y.to_excel(writer, sheet_name='합격_매연측정치_통계')
    gas_grp_y_vin.to_excel(writer, sheet_name='합격_매연측정치_차대번호_통계')
    gas_grp_n.to_excel(writer, sheet_name='불합격_매연측정치_통계')
    gas_grp_n_vin.to_excel(writer, sheet_name='불합격_매연측정치_차대번호_통계')
    # 합격/불합격 매연측정치 비교 통계
    pd.merge(gas_grp_y.reset_index(), gas_grp_n.reset_index(), on='배출가스인증번호', how='outer').to_excel(writer, sheet_name='합격_불합격_매연측정치_통계_비교')

C:\Users\kbjung\AppData\Local\Temp\ipykernel_9420\1178978199.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pd.merge(gas_grp_y.reset_index(), gas_grp_n.reset_index(), on='배출가스인증번호', how='outer').to_excel(writer, sheet_name='합격_불합격_매연측정치_통계_비교')


## 3등급, 5등급에 동일 배출가스인증번호 존재 여부 확인

In [145]:
total.columns

Index(['차대번호', '차량번호_x', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자',
       '검사유효일', '배출가스인증번호', '등급', '차명_x', '차종분류', '차종유형', '자동차형식', '제작사명',
       '연료', '엔진형식_x', '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명',
       '소유자구분', '말소여부', '일치유무', '사용연료', '주행거리', '차명_y', '차량번호_y', '차종', '차량연식',
       '엔진형식_y', '검사방법', '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값',
       '무부하매연판정1', '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6',
       '무부하매연허용치1', '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5',
       '무부하매연허용치6', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4',
       '무부하매연측정치5', '무부하매연측정치6', '정기정밀', 'vin10', 'vin_year'],
      dtype='object')

In [146]:
# about 59s
gasnm_list = []
car_y_list = []
vin10_y_list = []
for one in tqdm(total['배출가스인증번호'].unique()):
    temp = total[total['배출가스인증번호'] == one].reset_index(drop=True)
    try:
        cary_min = int(temp['차량연식'].min())
        cary_max = int(temp['차량연식'].max())
    except:
        cary_min = temp['차량연식'].min()
        cary_max = temp['차량연식'].max()

    car_y_list.append(f'{cary_min}-{cary_max}')
    try:
        vin10y_min = int(temp['vin_year'].min())
        vin10y_max = int(temp['vin_year'].max())
    except:
        vin10y_min = temp['vin_year'].min()
        vin10y_max = temp['vin_year'].max()
    vin10_y_list.append(f'{vin10y_min}-{vin10y_max}')
    
    gasnm_list.append(one)
len(gasnm_list), len(car_y_list), len(vin10_y_list)

100%|██████████| 206/206 [00:16<00:00, 12.70it/s]


(206, 206, 206)

In [147]:
year_df = pd.DataFrame({'배출가스인증번호':gasnm_list, '차량연식_범위':car_y_list, 'vin10자리연식_범위':vin10_y_list})
year_df.shape

(206, 3)

In [148]:
# about 16s
gas3 = g3['배출가스인증번호'].unique()
gas5 = g5['배출가스인증번호'].unique()

gasnm_list = []
gas3_jud_list = []
gas5_jud_list = []
gas3_num_car_list = []
gas5_num_car_list = []

for one in tqdm(total['배출가스인증번호'].unique()):
    if one in gas3:
        gas3_jud_list.append('Y')
        n3 = g3[g3['배출가스인증번호'] == one].shape[0]
        gas3_num_car_list.append(n3)
    else:
        gas3_jud_list.append('N')
        gas3_num_car_list.append(0)
    if one in gas5:
        gas5_jud_list.append('Y')
        n5 = g5[g5['배출가스인증번호'] == one].shape[0]
        gas5_num_car_list.append(n5)
    else:
        gas5_jud_list.append('N')
        gas5_num_car_list.append(0)
    gasnm_list.append(one)
len(gasnm_list), len(gas3_jud_list), len(gas3_num_car_list), len(gas5_num_car_list), len(gas5_jud_list)

100%|██████████| 206/206 [00:16<00:00, 12.63it/s]


(206, 206, 206, 206, 206)

In [149]:
gas35_jud_df = pd.DataFrame({'배출가스인증번호': gasnm_list, '3등급에_존재':gas3_jud_list, '5등급에_존재':gas5_jud_list, '3등급에_존재_차량_대수':gas3_num_car_list, '5등급에_존재_차량_대수':gas5_num_car_list})
gas35_jud_df.head()

,배출가스인증번호,3등급에_존재,5등급에_존재,3등급에_존재_차량_대수,5등급에_존재_차량_대수
0,9MY-KM-14-41,N,Y,0,36
1,7MY-SY-14-60,N,N,0,0
2,9MY-KM-14-22,Y,N,87217,0
3,nan,N,N,0,0
4,7MY-HD-14-47,N,N,0,0


In [150]:
g4_gasnm_grp = total.groupby(['배출가스인증번호'], as_index=False)['차대번호'].count()
g4_gasnm_grp.head()

,배출가스인증번호,차대번호
0,4MY-AD-20,29
1,4MY-DC-33,7
2,4MY-PA-06,13
3,5MY-AD-13-06,348
4,5MY-AD-13-07,118


In [151]:
g4_gasnm_info1 = g4_gasnm_grp.merge(year_df, on='배출가스인증번호', how='left')
g4_gasnm_info1.shape

(206, 4)

In [152]:
g4_gasnm_info2 = g4_gasnm_info1.merge(gas35_jud_df, on='배출가스인증번호', how='left')
g4_gasnm_info2.shape

(206, 8)

### [출력] 4등급과_3등급_5등급_동일_배출가스인증번호_대수

In [153]:
g4_gasnm_info2.to_excel('analysis2/4등급_데이터_점검/[통계][배출가스인증번호별]4등급과_3등급_5등급_동일_배출가스인증번호_대수.xlsx', index=False)

### 배인번호별 추가 컬럼
- 제작사, 엔진형식, 3등급에 존재 차량대수, 5등급에 존재 차량 대수, DPF 유무

#### [로드] euro4

In [154]:
euro = pd.read_excel(os.path.join('D:/data/big2', 'dpf.xlsx'))
euro

,배출가스인증번호,DPF유무
0,5MY-HD-24-59,유
1,6MY-HD-24-100,유
2,7MY-HD-24-57,무
3,7MY-HD-13-58,유
4,5MY-HD-24-35,무
...,...,...
77,8MY-FK-14-1,유
78,8MY-FK-14-2,유
79,5MY-HB-14-1,유
80,5MY-HB-14-4,유


In [155]:
tot1 = total.merge(euro, on='배출가스인증번호', how='left')
tot1['DPF유무'] = tot1['DPF유무'].fillna('무')
tot1['DPF유무'].value_counts(dropna=True)

무    857433
유    296380
Name: DPF유무, dtype: int64

In [156]:
tot1.shape

(1153813, 64)

In [157]:
tot1.columns

Index(['차대번호', '차량번호_x', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자',
       '검사유효일', '배출가스인증번호', '등급', '차명_x', '차종분류', '차종유형', '자동차형식', '제작사명',
       '연료', '엔진형식_x', '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명',
       '소유자구분', '말소여부', '일치유무', '사용연료', '주행거리', '차명_y', '차량번호_y', '차종', '차량연식',
       '엔진형식_y', '검사방법', '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값',
       '무부하매연판정1', '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6',
       '무부하매연허용치1', '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5',
       '무부하매연허용치6', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4',
       '무부하매연측정치5', '무부하매연측정치6', '정기정밀', 'vin10', 'vin_year', 'DPF유무'],
      dtype='object')

In [158]:
# about 16s
total_ss = pd.DataFrame()
for one in tqdm(g4_gasnm_info2['배출가스인증번호'].unique()):
    s1 = g4_gasnm_info2[g4_gasnm_info2['배출가스인증번호'] == one]
    temp = tot1[tot1['배출가스인증번호'] == one]
    s2 = pd.DataFrame(temp['제작사명'].dropna().unique())
    s3 = pd.DataFrame(temp['엔진형식_x'].dropna().unique())
    s4 = pd.DataFrame(temp['DPF유무'].dropna().unique())
    ss = pd.concat([s1, s2, s3, s4], axis=1)
    total_ss = pd.concat([total_ss, ss], ignore_index=True)
total_ss.shape

100%|██████████| 206/206 [00:17<00:00, 12.00it/s]


(1069, 11)

In [159]:
total_ss.head()

,배출가스인증번호,차대번호,차량연식_범위,vin10자리연식_범위,3등급에_존재,5등급에_존재,3등급에_존재_차량_대수,5등급에_존재_차량_대수,0,0,0
0,4MY-AD-20,29.0,2006-2006,2006-2006,N,N,0.0,0.0,아우디폭스바겐코리아(주),BMK,무
1,4MY-DC-33,7.0,2007-2007,2007-2007,N,Y,0.0,1502.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,타타대우상용차(주),DV11,무
3,4MY-PA-06,13.0,2005-2006,2005-2006,N,N,0.0,0.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(주)볼보자동차코리아,204D,무


In [160]:
total_ss.tail()

,배출가스인증번호,차대번호,차량연식_범위,vin10자리연식_범위,3등급에_존재,5등급에_존재,3등급에_존재_차량_대수,5등급에_존재_차량_대수,0,0,0
1064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,642 993,NaN
1065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C12,NaN
1066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M9R,NaN
1067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z20S,NaN
1068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D2676 LOH02,NaN


In [161]:
total_ss.columns

Index([     '배출가스인증번호',          '차대번호',       '차량연식_범위',  'vin10자리연식_범위',
             '3등급에_존재',       '5등급에_존재', '3등급에_존재_차량_대수', '5등급에_존재_차량_대수',
                     0,               0,               0],
      dtype='object')

In [162]:
total_ss.columns = ['배출가스인증번호', '차대번호', '차량연식_범위', 'vin10자리연식_범위', '3등급에_존재', '5등급에_존재', '3등급에_존재_차량_대수', '5등급에_존재_차량_대수', '제작사명', '엔진형식', 'DPF유무']

In [163]:
total_ss.columns

Index(['배출가스인증번호', '차대번호', '차량연식_범위', 'vin10자리연식_범위', '3등급에_존재', '5등급에_존재',
       '3등급에_존재_차량_대수', '5등급에_존재_차량_대수', '제작사명', '엔진형식', 'DPF유무'],
      dtype='object')

#### [출력] [통계][배출가스인증번호별]4등급과_3등급_5등급_동일_배출가스인증번호_대수_제작사_엔진형식_DPF유무

In [164]:
total_ss.to_excel('analysis2/4등급_데이터_점검/[통계][배출가스인증번호별]4등급과_3등급_5등급_동일_배출가스인증번호_대수_제작사_엔진형식_DPF유무.xlsx', index=False)

# CODE End